### Proxy target variable engineering with RFM and clustering

#### Compute RFM metrics per customer

In [1]:
# --- Step 1: Load raw transactions and ensure datetime ---
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

# Adjust path as needed
df_raw = pd.read_csv("/content/data.csv")

# Robust datetime parsing
df_raw["TransactionStartTime"] = pd.to_datetime(
    df_raw["TransactionStartTime"], errors="coerce"
)

# Drop rows without a valid datetime or CustomerId/Amount for RFM integrity
df_raw = df_raw.dropna(subset=["TransactionStartTime", "CustomerId", "Amount"])

# --- Step 2: Snapshot date for consistent Recency ---
snapshot_date = df_raw["TransactionStartTime"].max()
if pd.isna(snapshot_date):
    raise ValueError("No valid TransactionStartTime values found to compute RFM.")
snapshot_date = snapshot_date + pd.Timedelta(days=1)

# --- Step 3: RFM aggregation (one row per CustomerId) ---
rfm = df_raw.groupby("CustomerId").agg(
    Recency=("TransactionStartTime", lambda x: (snapshot_date - x.max()).days),
    Frequency=("TransactionStartTime", "count"),
    Monetary=("Amount", "sum"),
).reset_index()

# Guard against missing/invalid numeric values
rfm["Recency"] = pd.to_numeric(rfm["Recency"], errors="coerce").fillna(rfm["Recency"].median())
rfm["Frequency"] = pd.to_numeric(rfm["Frequency"], errors="coerce").fillna(0)
rfm["Monetary"] = pd.to_numeric(rfm["Monetary"], errors="coerce").fillna(0)

print("RFM head:\n", rfm.head())
print("RFM shape:", rfm.shape)


RFM head:
         CustomerId  Recency  Frequency  Monetary
0     CustomerId_1       35          1  -10000.0
1    CustomerId_10       35          1  -10000.0
2  CustomerId_1001       41          5   20000.0
3  CustomerId_1002       16          8    3325.0
4  CustomerId_1004        4          1    2000.0
RFM shape: (1952, 4)


#### Compute RFM metrics per customer

In [2]:
# --- Step 4: Scale RFM features ---
rfm_features = rfm[["Recency", "Frequency", "Monetary"]].copy()
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)

# --- Step 5: K-Means clustering (3 clusters, reproducible) ---
kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
rfm["Cluster"] = kmeans.fit_predict(rfm_scaled)

# --- Step 6: Cluster profiling to identify high-risk segment ---
cluster_summary = (
    rfm.groupby("Cluster")[["Recency", "Frequency", "Monetary"]].mean().sort_index()
)
print("Cluster summary (means):\n", cluster_summary)

# Define high-risk as least engaged: low Frequency and low Monetary (tie-break by high Recency)
ranked = cluster_summary.assign(
    freq_rank=cluster_summary["Frequency"].rank(method="min", ascending=True),
    mon_rank=cluster_summary["Monetary"].rank(method="min", ascending=True),
    rec_rank=cluster_summary["Recency"].rank(method="min", ascending=False),
)
ranked["risk_score"] = ranked["freq_rank"] + ranked["mon_rank"] + ranked["rec_rank"]
high_risk_cluster = ranked["risk_score"].idxmin()

# Assign binary target
rfm["is_high_risk"] = (rfm["Cluster"] == high_risk_cluster).astype(int)
print("Chosen high-risk cluster:", high_risk_cluster)
print("Target distribution:", rfm["is_high_risk"].value_counts().to_dict())


Cluster summary (means):
            Recency    Frequency      Monetary
Cluster                                      
0         6.519969    26.672670  1.883884e+05
1        13.000000  2075.000000 -5.328000e+07
2        29.348665     7.059347  5.899985e+04
Chosen high-risk cluster: 2
Target distribution: {0: 1278, 1: 674}


#### Merge target back into Task 3 features and save

In [4]:
# --- Step 7: Load Task 3 processed features ---
processed_df = pd.read_csv("/content/processed_task3 (2).csv")

# Handle passthrough naming from ColumnTransformer (remainder__CustomerId)
if "CustomerId" not in processed_df.columns:
    if "remainder__CustomerId" in processed_df.columns:
        processed_df = processed_df.rename(columns={"remainder__CustomerId": "CustomerId"})
    else:
        raise KeyError("CustomerId column not found in processed_task3.csv. Ensure Task 3 preserved IDs.")

# Ensure CustomerId types align for merging
rfm["CustomerId"] = pd.to_numeric(rfm["CustomerId"], errors="coerce")
processed_df["CustomerId"] = pd.to_numeric(processed_df["CustomerId"], errors="coerce")

# Drop any rows with missing IDs before merge
rfm = rfm.dropna(subset=["CustomerId"])
processed_df = processed_df.dropna(subset=["CustomerId"])

# --- Step 8: Merge proxy target into processed features ---
final_df = processed_df.merge(
    rfm[["CustomerId", "is_high_risk"]],
    on="CustomerId",
    how="left"
)

# Sanity checks
print("is_high_risk NaNs after merge:", int(final_df["is_high_risk"].isna().sum()))
print("Rows before dropping NaNs:", final_df.shape[0])

# Drop rows without target to ensure model-ready data
final_df = final_df.dropna(subset=["is_high_risk"])

# Convert to integer type explicitly
final_df["is_high_risk"] = final_df["is_high_risk"].astype(int)

print("Rows after dropping NaNs:", final_df.shape[0])
print("Final target distribution:", final_df["is_high_risk"].value_counts().to_dict())

# --- Step 9: Save Task 4 dataset ---
final_df.to_csv("/content/processed_task4.csv", index=False)
print("Saved /content/processed_task4.csv")


is_high_risk NaNs after merge: 0
Rows before dropping NaNs: 0
Rows after dropping NaNs: 0
Final target distribution: {}
Saved /content/processed_task4.csv
